**Import Library yang Diperlukan**

Kode ini mengimpor beberapa library penting yang digunakan dalam proses pengolahan data dan pengembangan model. `base64` dan `json` diimpor untuk pengkodean dan manipulasi data berbasis JSON, sedangkan `random` digunakan untuk operasi yang melibatkan pemilihan acak. `pandas` diimpor untuk manipulasi dan analisis data dalam bentuk tabel, `requests` digunakan untuk melakukan permintaan HTTP, dan `tensorflow` diimpor sebagai library utama untuk pengembangan dan pelatihan model machine learning.

In [162]:
# Cell 1: Import library yang diperlukan
import base64
import json
import random
import pandas as pd
import requests
import tensorflow as tf

**Membaca Data dari File CSV**

Kode ini menggunakan `pandas` untuk membaca data dari file CSV yang terletak di path `"C:\Users\ANANG\MLOps2\Submission_2\data\heart.csv"`. Hasil pembacaan ini disimpan dalam variabel `data_csv`, yang menjadi sebuah DataFrame `pandas` yang memungkinkan manipulasi dan analisis data lebih lanjut.

In [163]:
# Membaca data dari file CSV
data_csv = pd.read_csv(r"C:\Users\ANANG\MLOps2\Submission_2\data\heart.csv")

**Memilih Data Acak untuk Prediksi**

Kode ini memilih satu baris data secara acak dari DataFrame `data_csv` untuk digunakan sebagai input prediksi. Pertama, kode mendapatkan daftar nama kolom dari DataFrame dan memilih indeks baris acak menggunakan `random.randint()`. Kemudian, kode mengambil nilai fitur dari baris yang dipilih dan membuat dictionary `inputs` di mana setiap nama kolom dipetakan ke nilai fitur yang sesuai. Dictionary `inputs` ini siap digunakan sebagai input untuk model prediksi.

In [164]:
# Cell 2: Memilih data acak untuk prediksi
columns = data_csv.columns.values
rand = random.randint(0, len(data_csv) - 1)

# Mengambil fitur
features = data_csv.values[rand]

# Membuat dictionary input
inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Age': 54,
 'Sex': 'M',
 'ChestPainType': 'ASY',
 'RestingBP': 136,
 'Cholesterol': 220,
 'FastingBS': 0,
 'RestingECG': 'Normal',
 'MaxHR': 140,
 'ExerciseAngina': 'Y',
 'Oldpeak': 3.0,
 'ST_Slope': 'Flat',
 'HeartDisease': 1}

**Mendefinisikan Fungsi-Fungsi Helper untuk Membuat TFRecord**

Kode ini mendefinisikan tiga fungsi helper yang masing-masing digunakan untuk mengonversi nilai ke dalam format yang sesuai untuk disimpan dalam TFRecord. TFRecord adalah format yang efisien untuk menyimpan data terstruktur dalam TensorFlow. 

1. **`string_feature(value)`**: Fungsi ini menerima nilai string dan mengonversinya menjadi `tf.train.Feature` dengan tipe `bytes_list`.
2. **`float_feature(value)`**: Fungsi ini menerima nilai float dan mengonversinya menjadi `tf.train.Feature` dengan tipe `float_list`.
3. **`int_feature(value)`**: Fungsi ini menerima nilai integer dan mengonversinya menjadi `tf.train.Feature` dengan tipe `int64_list`.

Ketiga fungsi ini membantu dalam membangun protokol buffer yang sesuai untuk menulis data ke dalam file TFRecord.

In [165]:
# Cell 3: Mendefinisikan fungsi-fungsi helper untuk membuat TFRecord
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

**Fungsi untuk Menyiapkan JSON Request**

Kode ini mendefinisikan fungsi `prepare_json` yang bertujuan untuk menyiapkan request JSON yang akan dikirim ke model TensorFlow Serving. Berikut langkah-langkah yang dilakukan oleh fungsi ini:

1. **Inisialisasi `feature_spec`**:
   - `feature_spec` adalah dictionary yang akan menyimpan fitur dalam format yang sesuai untuk TFRecord.

2. **Iterasi Melalui `inputs`**:
   - Fungsi ini iterasi melalui setiap pasangan kunci (nama fitur) dan nilai dalam dictionary `inputs`.
   - Berdasarkan tipe nilai (string, float, atau integer), nilai tersebut dikonversi menggunakan fungsi helper yang telah didefinisikan (`string_feature`, `float_feature`, `int_feature`) dan ditambahkan ke `feature_spec`.

3. **Membangun `tf.train.Example`**:
   - `feature_spec` digunakan untuk membangun objek `tf.train.Example`, yang kemudian diserialisasi menjadi string biner menggunakan metode `SerializeToString`.

4. **Mengkodekan `tf.train.Example` dalam Base64**:
   - Hasil serialisasi di-encode menggunakan base64 dan dimasukkan ke dalam format yang diharapkan oleh TensorFlow Serving.

5. **Membuat JSON Request**:
   - Hasil akhir adalah dictionary yang mencakup `signature_name` dan `instances`. Dictionary ini dikonversi menjadi string JSON menggunakan `json.dumps`.

Fungsi ini mengembalikan string JSON yang siap untuk dikirim sebagai request ke TensorFlow Serving, dengan data input yang di-encode sesuai spesifikasi model.

In [166]:
# Cell 4: Fungsi untuk menyiapkan JSON request
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        else:
            feature_spec[keys] = int_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

**Fungsi untuk Melakukan Prediksi**

Kode ini mendefinisikan fungsi `make_predictions` yang bertujuan untuk mengirim data input ke model TensorFlow Serving dan mendapatkan hasil prediksi. Berikut langkah-langkah yang dilakukan oleh fungsi ini:

1. **Mempersiapkan JSON Data**:
   - `json_data = prepare_json(inputs)`: Fungsi ini memanggil `prepare_json` untuk mengonversi data input menjadi format JSON yang sesuai untuk dikirim ke endpoint model.

2. **Menentukan Endpoint Model**:
   - `endpoint = "https://mlops-production-6fea.up.railway.app/v1/models/heart-disease-model:predict"`: Endpoint ini adalah URL dari model TensorFlow Serving yang di-deploy untuk prediksi penyakit jantung.

3. **Mengirim Request Prediksi**:
   - `response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})`: Mengirimkan request POST ke endpoint model dengan data JSON yang telah dipersiapkan dan header yang menyatakan konten berupa JSON.

4. **Mendapatkan Hasil Prediksi**:
   - `prediction = response.json()["predictions"][0][0]`: Mendapatkan hasil prediksi dari response JSON yang dikembalikan oleh model. Hasil prediksi ini diambil dari field `predictions`.

5. **Menginterpretasikan Hasil Prediksi**:
   - Fungsi ini menginterpretasikan hasil prediksi: jika nilai prediksi kurang dari 0.5, fungsi mengembalikan "Tidak Mengalami Penyakit Jantung", sebaliknya jika nilai prediksi 0.5 atau lebih, fungsi mengembalikan "Mengalami Penyakit Jantung".

Fungsi `make_predictions` ini mengautomasi proses pengiriman data ke model yang telah di-deploy, mendapatkan hasil prediksi, dan menginterpretasikan hasil tersebut dalam bentuk yang mudah dipahami.

In [167]:
# Cell 5: Fungsi untuk melakukan prediksi
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://mlops-production-6fea.up.railway.app/v1/models/heart-disease-model:predict"
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.5:
        return "Tidak Mengalami Penyakit Jantung"
    else:
        return "Mengalami Penyakit Jantung"

**Melakukan Prediksi dan Menampilkan Hasil**

Kode ini menjalankan fungsi `make_predictions` dengan data input yang telah dipersiapkan, kemudian menampilkan hasil prediksi dan fitur input yang digunakan. Berikut langkah-langkah yang dilakukan:

1. **Melakukan Prediksi**:
   - `hasil_prediksi = make_predictions(inputs)`: Memanggil fungsi `make_predictions` dengan dictionary `inputs` yang berisi fitur-fitur input, dan menyimpan hasil prediksinya dalam variabel `hasil_prediksi`.

2. **Menampilkan Hasil Prediksi**:
   - `print(f'Hasil Prediksi: {hasil_prediksi}')`: Mencetak hasil prediksi ke konsol, yang akan berupa "Tidak Mengalami Penyakit Jantung" atau "Mengalami Penyakit Jantung" berdasarkan nilai prediksi yang dikembalikan oleh model.

3. **Menampilkan Fitur Input**:
   - `print('Fitur Input:', inputs)`: Mencetak dictionary `inputs` yang berisi fitur-fitur yang digunakan untuk prediksi ke konsol, sehingga pengguna dapat melihat data input yang digunakan.

Kode ini memungkinkan pengguna untuk menjalankan prediksi menggunakan model yang di-deploy dan melihat hasil prediksi serta data input yang mendasarinya.

In [168]:
# Cell 6: Melakukan prediksi dan menampilkan hasil
hasil_prediksi = make_predictions(inputs)
print(f'Hasil Prediksi: {hasil_prediksi}')
print('Fitur Input:', inputs)

Hasil Prediksi: Mengalami Penyakit Jantung
Fitur Input: {'Age': 54, 'Sex': 'M', 'ChestPainType': 'ASY', 'RestingBP': 136, 'Cholesterol': 220, 'FastingBS': 0, 'RestingECG': 'Normal', 'MaxHR': 140, 'ExerciseAngina': 'Y', 'Oldpeak': 3.0, 'ST_Slope': 'Flat', 'HeartDisease': 1}
